## Grupo

227124 Jorge Henrique Monteiro dos Santos

262251 Vitor Mello de Araujo Lima

## Objetivo

Esta tarefa busca  ...


In [26]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from pandas import DataFrame

# Leia

A leitura dos dados é relizada utilizando a biblioteca Pandas. Como os dados são separados por espaço em branco, é necesasario especificar o parametro sep. Além disso, como a primeira linha não faz parte do conjunto dados e não há um linha com os nomes dos atributos, foi especificado o parametro names e o header.


In [11]:
url = "https://www.ic.unicamp.br/~wainer/cursos/1s2021/432/solar-flare.csv"
names = ["Class","Largest spot size","Spot distribution", "Activity", "Evolution", "Prv 24h flare activity", "Historically-complex", 
        "Complexixity change in sun's disk", "Area", "Largest Spot Area", "C-Class", "M-Class", "X-Class" ]
data = pd.read_csv(url,sep = " ", header = 0, names = names)

In [12]:
data.shape

(1066, 13)

## Primeira linha

In [16]:
data.head(1)

,Class,Largest spot size,Spot distribution,Activity,Evolution,Prv 24h flare activity,Historically-complex,Complexixity change in sun's disk,Area,Largest Spot Area,C-Class,M-Class,X-Class
0,H,A,X,1,3,1,1,1,1,1,0,0,0


## Ultima Linha

In [18]:
data.tail(1)

,Class,Largest spot size,Spot distribution,Activity,Evolution,Prv 24h flare activity,Historically-complex,Complexixity change in sun's disk,Area,Largest Spot Area,C-Class,M-Class,X-Class
1065,B,X,O,1,1,1,1,2,1,1,0,0,0


# Converta os atributos categóricos para numéricos

In [25]:
Class = pd.get_dummies(data['Class'], prefix='Class')
#print(Class.head())

largSpotSize = pd.get_dummies(data['Largest spot size'], prefix='Larg_spot_size')
#print(largSpotSize.head())

spotDistr = pd.get_dummies(data['Spot distribution'], prefix='Spot_distr')
#print(spotDistr.head())

data_num = pd.concat([data,Class,largSpotSize,spotDistr], axis=1)
data_num.drop(['Class', 'Largest spot size','Spot distribution' ],axis=1, inplace=True)

data_num.head()

,Activity,Evolution,Prv 24h flare activity,Historically-complex,Complexixity change in sun's disk,Area,Largest Spot Area,C-Class,M-Class,X-Class,...,Larg_spot_size_A,Larg_spot_size_H,Larg_spot_size_K,Larg_spot_size_R,Larg_spot_size_S,Larg_spot_size_X,Spot_distr_C,Spot_distr_I,Spot_distr_O,Spot_distr_X
0,1,3,1,1,1,1,1,0,0,0,...,1,0,0,0,0,0,0,0,0,1
1,1,3,1,1,2,1,1,0,0,0,...,0,0,0,1,0,0,0,0,1,0
2,1,3,1,1,2,1,1,0,0,0,...,0,0,0,0,1,0,0,0,1,0
3,1,2,1,1,1,1,1,0,0,0,...,0,0,0,1,0,0,0,0,0,1
4,1,1,1,1,2,1,1,0,0,0,...,0,0,0,0,1,0,0,0,0,1


# Centering and scaling

In [29]:
from sklearn.preprocessing import StandardScaler

data_scaled = DataFrame(StandardScaler().fit_transform(data_num), columns=data_num.columns)
data_scaled.head()

,Activity,Evolution,Prv 24h flare activity,Historically-complex,Complexixity change in sun's disk,Area,Largest Spot Area,C-Class,M-Class,X-Class,...,Larg_spot_size_A,Larg_spot_size_H,Larg_spot_size_K,Larg_spot_size_R,Larg_spot_size_S,Larg_spot_size_X,Spot_distr_C,Spot_distr_I,Spot_distr_O,Spot_distr_X
0,-0.426401,0.964865,-0.184586,-0.823857,-2.648592,-0.161203,0.0,-0.359337,-0.154969,-0.06511,...,1.983730,-0.161203,-0.212363,-0.507026,-0.796850,-0.396784,-0.184249,-0.514326,-0.899915,1.490149
1,-0.426401,0.964865,-0.184586,-0.823857,0.377559,-0.161203,0.0,-0.359337,-0.154969,-0.06511,...,-0.504101,-0.161203,-0.212363,1.972285,-0.796850,-0.396784,-0.184249,-0.514326,1.111216,-0.671074
2,-0.426401,0.964865,-0.184586,-0.823857,0.377559,-0.161203,0.0,-0.359337,-0.154969,-0.06511,...,-0.504101,-0.161203,-0.212363,-0.507026,1.254942,-0.396784,-0.184249,-0.514326,1.111216,-0.671074
3,-0.426401,-0.647276,-0.184586,-0.823857,-2.648592,-0.161203,0.0,-0.359337,-0.154969,-0.06511,...,-0.504101,-0.161203,-0.212363,1.972285,-0.796850,-0.396784,-0.184249,-0.514326,-0.899915,1.490149
4,-0.426401,-2.259418,-0.184586,-0.823857,0.377559,-0.161203,0.0,-0.359337,-0.154969,-0.06511,...,-0.504101,-0.161203,-0.212363,-0.507026,1.254942,-0.396784,-0.184249,-0.514326,-0.899915,1.490149
